In [1]:
import pandas as pd
import numpy as np
import re

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
ETF_IR = pd.read_csv(
    "/content/drive/MyDrive/DDDM_Project/Data/Covariates/ETF+IR.csv"
)


etf_list = sorted(ETF_IR["ETF"].unique().tolist())
len(etf_list  )

578

In [ ]:
import yfinance as yf

tickers = etf_list

rows = []
for t in tickers:
    info = yf.Ticker(t).info
    rows.append({
        "ETF": t,
        "shortName": info.get("shortName"),
        "longName": info.get("longName"),
        "quoteType": info.get("quoteType"),
        "category": info.get("category"),
        "fundFamily": info.get("fundFamily"),
        "country": info.get("country"),
    })

meta = pd.DataFrame(rows)
meta.head()

Exception ignored from cffi callback <function buffer_callback at 0x7b6acebec7c0>:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/curl_cffi/curl.py", line 100, in buffer_callback
    @ffi.def_extern()
    
KeyboardInterrupt: 


RequestException: Failed to perform, curl: (23) Failure writing output to destination, passed 13 returned 0. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.

In [11]:
meta=pd.read_csv("/content/drive/MyDrive/DDDM_Project/Data/Covariates/meta.csv")

In [20]:
for c in ["category", "shortName", "longName"]:
    if c not in meta.columns:
        meta[c] = ""
    meta[c] = meta[c].fillna("").astype(str)

text = (meta["category"] + " " + meta["shortName"] + " " + meta["longName"]).str.lower()

# -----------------------------
# Keyword sets (unambiguous, minimal overlap)
# -----------------------------
# GOV bonds (sovereign/rates)
gov_kw = (
    r"\b(treasury|treasuries|t[- ]?bill|t[- ]?note|t[- ]?bond|sovereign|government bond)\b"
    r"|\b(tips|inflation[- ]?protected)\b"
)

# CREDIT / spread risk bonds (corporate/muni/structured/loans/etc.)
credit_kw = (
    r"\b(corporate bond|investment grade|high yield|junk bond)\b"
    r"|\b(credit|spread)\b"
    r"|\b(muni|municipal)\b"
    r"|\b(bank loan|leveraged loan|loan)\b"
    r"|\b(floating[- ]?rate|frn|floating rate note)\b"
    r"|\b(preferred)\b"
    r"|\b(securitized|mortgage[- ]?backed|mbs|abs|cmo|cmbs)\b"
    r"|\b(emerging markets bond|multisector bond)\b"
)

# MACRO (commodities + FX combined)
# Avoid specific currency names (USD/EUR/JPY) because "USD-denominated bonds" is common.
macro_kw = (
    # FX explicit language only
    r"\b(currency|currencies|forex|fx|foreign exchange|currency hedg(e|ed|ing)|currency strategy|carry)\b"
    # Commodities / real assets
    r"|\b(commodit(y|ies))\b"
    r"|\b(gold|silver|platinum|palladium)\b"
    r"|\b(crude|oil|wti|brent|natural gas|gasoline)\b"
    r"|\b(agriculture|corn|wheat|soy(bean)?s?|sugar|coffee|cocoa|cotton)\b"
    r"|\b(industrial metals|copper|aluminum|nickel|zinc)\b"
    r"|\b(uranium)\b"
)

# Bond context exclusion for FX-ish words inside bond products (e.g., "currency hedged bond")
# NOTE: We do NOT exclude macro from bonds because commodity-linked notes can exist;
# but we do want to avoid mis-tagging bond funds as "macro" purely due to "USD-denominated" etc.
# Since we already excluded currency names, this is mainly for cases like "currency hedged bond"
bond_context_kw = (
    r"\b(bond|bonds|loan|loans|debt|credit|corporate|treasury|muni|municipal|investment grade|high yield)\b"
    r"|\b(floating[- ]?rate|frn|securitized|mbs|abs|cmo|cmbs)\b"
)

# EQUITY: strong equity signals (avoid ambiguous "core")
equity_kw = (
    r"\b(equity|equities|stock|stocks)\b"
    r"|\b(s&p|msci|russell|nasdaq|dow)\b"
    r"|\b(dividend|growth|value|momentum|quality|small[- ]?cap|mid[- ]?cap|large[- ]?cap)\b"
    r"|\b(communication services|technology|health care|financials|industrials|energy|materials|utilities|consumer( discretionary| staples)?)\b"
    r"|\b(reit|real estate)\b"
)

# INTERNATIONAL (descriptive, orthogonal)
intl_kw = (
    r"\b(international|global|world|foreign)\b"
    r"|\b(ex[- ]?us|non[- ]?us)\b"
    r"|\b(emerging|developed|eafe|frontier)\b"
    r"|\b(china|japan|europe|pacific|asia|latam|eurozone|uk)\b"
)

# -----------------------------
# Raw hits
# -----------------------------
hit_gov    = text.str.contains(gov_kw, regex=True)
hit_credit = text.str.contains(credit_kw, regex=True)
hit_equity = text.str.contains(equity_kw, regex=True)
hit_intl   = text.str.contains(intl_kw, regex=True)

# Macro: to reduce false positives from "currency hedged bond", require macro_kw hit
# and NOT (bond context AND only FX words). We keep this simple:
hit_macro_raw = text.str.contains(macro_kw, regex=True)
hit_bond_ctx  = text.str.contains(bond_context_kw, regex=True)

# If it's a bond-context fund and only matched FX-language (not commodities), it may be "currency-hedged bonds".
# We'll treat that as macro=0 and send to needs_review via mixed tags if needed.
fx_only_kw = r"\b(currency|currencies|forex|fx|foreign exchange|currency hedg(e|ed|ing)|currency strategy|carry)\b"
hit_fx_only = text.str.contains(fx_only_kw, regex=True)
hit_commodity_part = text.str.contains(
    r"\b(commodit(y|ies)|gold|silver|platinum|palladium|crude|oil|wti|brent|natural gas|gasoline|agriculture|corn|wheat|soy(bean)?s?|sugar|coffee|cocoa|cotton|industrial metals|copper|aluminum|nickel|zinc|uranium)\b",
    regex=True
)

hit_macro = hit_macro_raw & ~(hit_bond_ctx & hit_fx_only & ~hit_commodity_part)

# -----------------------------
# 5 tags (macro combined) + needs_review
# -----------------------------
meta["is_gov"] = hit_gov.astype(int)
meta["is_credit"] = hit_credit.astype(int)
meta["is_macro"] = hit_macro.astype(int)
meta["is_international"] = hit_intl.astype(int)

# Equity only if equity signal and no competing primary asset-type tags
meta["is_equity"] = (hit_equity & ~(hit_gov | hit_credit | hit_macro)).astype(int)

# -----------------------------
# needs_review rules (minimal, high-signal)
# -----------------------------
# 1) No primary type detected
no_primary_type = (
    (meta["is_equity"] == 0) &
    (meta["is_gov"] == 0) &
    (meta["is_credit"] == 0) &
    (meta["is_macro"] == 0)
)

# 2) Equity + bond conflict
equity_bond_conflict = (meta["is_equity"] == 1) & ((meta["is_gov"] == 1) | (meta["is_credit"] == 1))

# 3) Gov + credit mixed (optional but recommended to confirm)
gov_credit_mixed = (meta["is_gov"] == 1) & (meta["is_credit"] == 1)

meta["needs_review"] = (no_primary_type | equity_bond_conflict | gov_credit_mixed).astype(int)

# -----------------------------
# Final tags df
# -----------------------------
tags = meta[[
    "ETF",
    "is_equity",
    "is_international",
    "is_gov",
    "is_credit",
    "is_macro",
    "needs_review"
]].copy()

# -----------------------------
# needs_review_df compatible with your manual input loop
# -----------------------------
needs_review_df = meta.loc[
    meta["needs_review"] == 1,
    ["ETF", "category", "shortName", "longName"]
].drop_duplicates(subset=["ETF"]).reset_index(drop=True)

# -----------------------------
# Quick counts
# -----------------------------
print("Tag counts:")
print(tags[["is_equity","is_international","is_gov","is_credit","is_macro","needs_review"]].sum().sort_values(ascending=False))
print("needs_review count:", len(needs_review_df))

Tag counts:
is_equity           392
is_international    136
needs_review         86
is_credit            70
is_gov               16
is_macro             16
dtype: int64
needs_review count: 86


/tmp/ipython-input-1657584373.py:72: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hit_gov    = text.str.contains(gov_kw, regex=True)
/tmp/ipython-input-1657584373.py:73: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hit_credit = text.str.contains(credit_kw, regex=True)
/tmp/ipython-input-1657584373.py:74: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hit_equity = text.str.contains(equity_kw, regex=True)
/tmp/ipython-input-1657584373.py:75: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  hit_intl   = text.str.contains(intl_kw, regex=True)
/tmp/ipython-input-1657584373.py:79: UserWarning: This pattern is interpreted as a regular expression

In [21]:
tags.head()

,ETF,is_equity,is_international,is_gov,is_credit,is_macro,needs_review
0,AFTY,0,1,0,0,0,1
1,AGZ,0,0,0,0,0,1
2,AIRR,1,0,0,0,0,0
3,ANGL,0,0,0,1,0,0
4,AOA,0,1,0,0,0,1


In [33]:


needs_review_df = meta.loc[
    tags["needs_review"] == 1,
    ["ETF", "category", "shortName", "longName", "is_equity", "is_international", "is_gov", "is_credit", "is_macro"]
].reset_index(drop=True)

needs_review_df

,ETF,category,shortName,longName,is_equity,is_international,is_gov,is_credit,is_macro
0,AFTY,China Region,Pacer CSOP FTSE China A50 ETF,Pacer CSOP FTSE China A50 ETF,0,1,0,0,0
1,AGZ,Short Government,iShares Agency Bond ETF,iShares Agency Bond ETF,0,0,0,0,0
2,AOA,Global Aggressive Allocation,iShares Core 80/20 Aggressive A,iShares Core 80/20 Aggressive Allocation ETF,0,1,0,0,0
3,ARKG,Health,ARK Genomic Revolution ETF,ARK Genomic Revolution ETF,0,0,0,0,0
4,BICK,Diversified Emerging Mkts,First Trust BICK Index Fund,First Trust BICK Index Fund,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
81,WINC,Short-Term Bond,Western Asset Short Duration In,Western Asset Short Duration Income ETF,0,0,0,0,0
82,WOMN,Large Blend,Impact Shares Women's Empowerme,Impact Shares Women’s Empowerment ETF,0,0,0,0,0
83,XCEM,Diversified Emerging Mkts,Columbia EM Core ex-China ETF,Columbia EM Core ex-China ETF,0,1,0,0,0
84,XLF,Financial,State Street Financial Select S,State Street Financial Select Sector SPDR ETF,0,0,0,0,0


In [36]:
review_etfs=needs_review_df[["ETF", "longName"]]
review_etfs.to_csv(
    "/content/drive/MyDrive/DDDM_Project/Data/needs_review_etfs.csv",
    index=False
)

In [34]:
needs_review_df.head(6)

,ETF,category,shortName,longName,is_equity,is_international,is_gov,is_credit,is_macro
0,AFTY,China Region,Pacer CSOP FTSE China A50 ETF,Pacer CSOP FTSE China A50 ETF,0,1,0,0,0
1,AGZ,Short Government,iShares Agency Bond ETF,iShares Agency Bond ETF,0,0,0,0,0
2,AOA,Global Aggressive Allocation,iShares Core 80/20 Aggressive A,iShares Core 80/20 Aggressive Allocation ETF,0,1,0,0,0
3,ARKG,Health,ARK Genomic Revolution ETF,ARK Genomic Revolution ETF,0,0,0,0,0
4,BICK,Diversified Emerging Mkts,First Trust BICK Index Fund,First Trust BICK Index Fund,0,1,0,0,0
5,BIV,Intermediate Core Bond,Vanguard Intermediate-Term Bond,Vanguard Intermediate-Term Bond Index Fund,0,0,0,0,0


In [40]:

SAVE_PATH = "/content/drive/MyDrive/DDDM_Project/Data/manual_tags_needs_review1.csv"
SAVE_EVERY = 5  # autosave every 10 ETFs

needs_review_df = needs_review_df.reset_index(drop=True)

TAGS = ["is_equity", "is_international", "is_gov", "is_credit", "is_macro"]

def prompt01(msg):
    while True:
        x = input(msg).strip()
        if x in ("0", "1"):
            return int(x)
        print("Enter 0 or 1 only.")

# Resume if you already started before
try:
    manual_tags_df = pd.read_csv(SAVE_PATH)
    done = set(manual_tags_df["ETF"].astype(str))
    rows = manual_tags_df.to_dict("records")
    print(f"Resuming: loaded {len(done)} already-tagged ETFs from {SAVE_PATH}")
except Exception:
    done = set()
    rows = []
    print("Starting new manual tagging session.")

total = len(needs_review_df)

for i, r in needs_review_df.iterrows():
    etf = str(r["ETF"])

    print("\n" + "="*70)
    print(f"[{i+1}/{total}] ETF: {etf}")

    for col in ["category", "shortName", "longName"]:
        if col in needs_review_df.columns:
            val = r.get(col, "")
            if isinstance(val, str) and val.strip():
                print(f"{col}: {val}")

    is_equity = prompt01("is_equity (1/0): ")
    is_international = prompt01("is_international (1/0): ")
    is_macro = prompt01("is_macro (1/0) [FX/currency] : ")
    is_gov = prompt01("is_gov (1/0): ")
    is_credit = prompt01("is_credit (1/0): ")

    rows.append({
        "ETF": etf,
        "is_equity": is_equity,
        "is_international": is_international,
        "is_gov": is_gov,
        "is_credit": is_credit,
        "is_macro": is_macro,
    })
    done.add(etf)

    # Autosave
    if len(rows) % SAVE_EVERY == 0:
        pd.DataFrame(rows).to_csv(SAVE_PATH, index=False)
        print(f"Autosaved {len(rows)} rows → {SAVE_PATH}")

manual_tags_df = (
    pd.DataFrame(rows)
      .drop_duplicates(subset=["ETF"], keep="last")
      .reset_index(drop=True)
)
manual_tags_df.to_csv(SAVE_PATH, index=False)
print(f"\nDone. Saved {len(manual_tags_df)} rows → {SAVE_PATH}")
manual_tags_df.head()

Starting new manual tagging session.

[1/86] ETF: AFTY
category: China Region
shortName: Pacer CSOP FTSE China A50 ETF
longName: Pacer CSOP FTSE China A50 ETF
is_equity (1/0): 1
is_international (1/0): 1
is_macro (1/0) [FX/currency] : 0
is_gov (1/0): 0
is_credit (1/0): 0

[2/86] ETF: AGZ
category: Short Government
shortName: iShares  Agency Bond ETF
longName: iShares Agency Bond ETF
is_equity (1/0): 0
is_international (1/0): 0
is_macro (1/0) [FX/currency] : 0
is_gov (1/0): 1
is_credit (1/0): 0

[3/86] ETF: AOA
category: Global Aggressive Allocation
shortName: iShares Core 80/20 Aggressive A
longName: iShares Core 80/20 Aggressive Allocation ETF
is_equity (1/0): 1
is_international (1/0): 1
is_macro (1/0) [FX/currency] : 0
is_gov (1/0): 1
is_credit (1/0): 1

[4/86] ETF: ARKG
category: Health
shortName: ARK Genomic Revolution ETF
longName: ARK Genomic Revolution ETF
is_equity (1/0): 1
is_international (1/0): 1
is_macro (1/0) [FX/currency] : 0
is_gov (1/0): 0
is_credit (1/0): 0

[5/86] ETF

,ETF,is_equity,is_international,is_gov,is_credit,is_macro
0,AFTY,1,1,0,0,0
1,AGZ,0,0,0,0,0
2,AOA,1,1,0,0,0
3,ARKG,1,1,0,0,0
4,BICK,1,1,0,0,0


In [44]:
manually_tagged=pd.read_csv("/content/drive/MyDrive/DDDM_Project/Data/manually_tagged.csv")
manually_tagged.head()

,ETF,is_equity,is_international,is_gov,is_credit,is_macro
0,AFTY,1,1,0,0,0
1,AGZ,0,0,1,0,0
2,AOA,1,1,1,1,0
3,ARKG,1,1,0,0,0
4,BICK,1,1,0,0,0


In [45]:
tags.head()

,ETF,is_equity,is_international,is_gov,is_credit,is_macro,needs_review
0,AFTY,0,1,0,0,0,1
1,AGZ,0,0,0,0,0,1
2,AIRR,1,0,0,0,0,0
3,ANGL,0,0,0,1,0,0
4,AOA,0,1,0,0,0,1


In [49]:
TAG_COLS = ["is_equity", "is_international", "is_gov", "is_credit", "is_macro"]

for c in TAG_COLS:
    manually_tagged[c] = pd.to_numeric(manually_tagged[c], errors="coerce").astype("Int64")

# --- Merge (manual overrides auto) ---
merged = tags.merge(
    manually_tagged,
    on="ETF",
    how="left",
    suffixes=("", "_manual")
)

for c in TAG_COLS:
    merged[c] = merged[f"{c}_manual"].combine_first(merged[c])

# --- Drop manual columns explicitly ---
merged = merged.drop(columns=[f"{c}_manual" for c in TAG_COLS])

# --- Finalize ---
merged = merged.sort_values("ETF").reset_index(drop=True)

merged.head(10)

,ETF,is_equity,is_international,is_gov,is_credit,is_macro,needs_review
0,AFTY,1,1,0,0,0,1
1,AGZ,0,0,1,0,0,1
2,AIRR,1,0,0,0,0,0
3,ANGL,0,0,0,1,0,0
4,AOA,1,1,1,1,0,1
5,ARKG,1,1,0,0,0,1
6,ASHX,1,1,0,0,0,0
7,ATMP,1,0,0,0,0,0
8,BAPR,1,0,0,0,0,0
9,BBUS,1,0,0,0,0,0


In [50]:
tagged_final=merged

tag_counts = tagged_final[TAG_COLS].sum().astype(int)
tag_counts

,0
is_equity,449
is_international,147
is_gov,49
is_credit,101
is_macro,18


In [53]:
tagged_final.to_csv("/content/drive/MyDrive/DDDM_Project/Data/tagged_final.csv", index=False)

In [54]:
no_tag_mask = (tagged_final[TAG_COLS].sum(axis=1) == 0)

no_tag_etfs = tagged_final.loc[no_tag_mask, ["ETF"]]

len(no_tag_etfs), no_tag_etfs.head(20)

(0,
 Empty DataFrame
 Columns: [ETF]
 Index: [])